This notebook creates a Workflow RO crate for the BatchConvert tool at the root of the repo.  
The notebook could be extended to document a run with the workflow hence creating a WF Run RO crate procedure.

The notebook uses the general purpose python rocrate library to create the crate (and the underlying json file).  
Adding items to the crate follows the following event sequence : 
- create an item 
- add it to the crate
- link the item to other items in the crate

Some functions of the rocrate package take care of both creating an item and adding it to the crate (ex: add_workflow).  

In [ ]:
import rocrate
#print(rocrate.__path__)

from rocrate.rocrate import ROCrate
from rocrate.model   import ComputerLanguage
from rocrate.model.person import Person

['/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rocrate']


In [2]:
%pwd

'/Users/thomasl/Documents/repos/BatchConvert'

In [12]:
# Create a crate for the given directory
# Files and folder will be added manually
crate = ROCrate(source = "")

# crate.add_workflow("batchconvert", main=True, lang = "shell") # throws an error, currently lang has to be one of "cwl", "galaxy", "knime", "nextflow", "snakemake", "compss", "autosubmit"
# However one can also pass a ComputerLanguage object to lang, see https://github.com/ResearchObject/ro-crate-py/issues/218#issuecomment-2753694857
bash_url = "https://www.gnu.org/software/bash/"
bash = ComputerLanguage(crate, identifier="#bash", properties = {"name" : "Bash",
                                                                 "identifier": {"@id": bash_url},
                                                                 "url": {"@id": bash_url}
                                                                })
crate.add(bash) # need to add the item first then reference it
main_wf = crate.add_workflow("batchconvert", main=True, lang = bash) # type: ignore

# Try adding the subworkflows has part of the WF crate
# For a run, one could omit the one not used, or only mention the one used via a createAction
# Same process, create the entities and add them to the crate (done in one go by add_workflow here), then link them to the main wf
workflow_tiff = crate.add_workflow(source = "pff2ometiff.nf", 
                                   main = False,
                                   lang = "nextflow")

workflow_zarr = crate.add_workflow(source = "pff2omezarr.nf", 
                                   main = False,
                                   lang = "nextflow")

# Add description to the secondary workflow
desc_template = "Nextflow workflow executed when passing the argument (i.e converting to) '{}' as first argument to the BatchConvert utility."
workflow_tiff["description"] = desc_template.format("ometiff") # type: ignore
workflow_zarr["description"] = desc_template.format("omezarr") # type: ignore

main_wf["hasPart"] = [workflow_tiff, workflow_zarr]  # type: ignore

# Add the authors and institution potentially
# works but not added as author of the workflow
author = crate.add(Person(crate, identifier="https://orcid.org/0000-0001-9823-0581", properties={"Name" : "Bugra Özdemir"})) # the O with umlaut gets a unicode character code, which is OK in a UI it renders properly
#crate["Authors"] = author # nope

# Add the authors at the root of the dataset entity as expected by workflow hub
"""
for e in crate.get_entities():
    
    if e.id == "./" :
        root_dataset_entity = e
        break

else: # for/else : hit if the for loop does not break
    raise Exception("Could not find root Dataset entity")
"""
crate.root_dataset["author"] = [author]

# Save the crate in a new subdirectory, all files and directories listed in the json will get copied to the subdirectory
crate.write("test_crate") # write and write_crate are the same
#crate.write_crate("my_crate")

In [10]:
dir(crate)

['_ROCrate__add_parts',
 '_ROCrate__entity_map',
 '_ROCrate__init_from_tree',
 '_ROCrate__read',
 '_ROCrate__read_contextual_entities',
 '_ROCrate__read_data_entities',
 '_ROCrate__validate_suite',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_copy_unlisted',
 '_get_root_jsonld',
 'add',
 'add_action',
 'add_dataset',
 'add_directory',
 'add_file',
 'add_jsonld',
 'add_or_update_jsonld',
 'add_test_definition',
 'add_test_instance',
 'add_test_suite',
 'add_tree',
 'add_workflow',
 'arcp_base_uri',
 'contextual_entities',
 'creativeWorkStatus',
 'creator',
 'data_entities',
 'datePublished',
 'default_entities',
 'delete',
 'dereference',
 'description',
 'examples

In [18]:
for entity in crate.get_entities():
    print(entity)

dir(entity)

<./ Dataset>
<ro-crate-metadata.json CreativeWork>
<test-knime-workflow.knwf File>
<make-workflow.ipynb File>


['_Entity__id',
 '_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_default_type',
 '_empty',
 '_jsonld',
 'append_to',
 'as_jsonld',
 'canonical_id',
 'clear',
 'crate',
 'datePublished',
 'delete',
 'fetch_remote',
 'format_id',
 'get',
 'id',
 'items',
 'keys',
 'pop',
 'popitem',
 'properties',
 'record_size',
 'setdefault',
 'source',
 'type',
 'update',
 'validate_url',
 'values',
 'write']